In [1]:
pdbid = "193l"

In [2]:
import gzip
from urllib.request import urlopen

from pdbfixer import PDBFixer


def fixer_from_pdbid(id: str) -> PDBFixer:
    """
    Load a PDBFixer object from a PDBID.

    Differs from ``PDBFixer(pdbid=id)`` because this function downloads the PDB
    file in gzipped format, which reduces network usage, whereas the
    ``PDBFixer`` constructor downloads in uncompressed .PDB format.
    """
    with urlopen(f"https://files.rcsb.org/download/{id}.pdb.gz") as gzfile:
        pdbfile = gzip.GzipFile(fileobj=gzfile)
        fixer = PDBFixer(pdbfile=pdbfile)
    return fixer

In [3]:
fixer = fixer_from_pdbid(pdbid)

In [4]:
fixer.missingResidues = {}
fixer.removeHeterogens(keepWater=False)
fixer.findMissingAtoms()
fixer.addMissingAtoms()
fixer.addMissingHydrogens(pH=7.4)

In [5]:
import openmm

forcefield = openmm.app.ForceField("amber99sbildn.xml", "amber99_obc.xml")
system = forcefield.createSystem(
    fixer.topology,
    nonbondedMethod=openmm.app.CutoffPeriodic,
    nonbondedCutoff=1.6 * openmm.unit.nanometers,
    constraints=None,
    rigidWater=True,
)
integrator = openmm.LangevinMiddleIntegrator(
    300 * openmm.unit.kelvin,
    5.0 / openmm.unit.picoseconds,
    1.0 * openmm.unit.femtoseconds,
)
# integrator.setConstraintTolerance(0.00001)
simulation = openmm.app.Simulation(
    fixer.topology,
    system,
    integrator,
)

simulation.context.setPositions(fixer.positions)

print("Minimizing...")
simulation.minimizeEnergy(maxIterations=100000)

Minimizing...


In [6]:
from openmm.app import PDBFile

fnout = f"../openff/pablo/_tests/data/{pdbid}_prepared.pdb"

with open(fnout, mode="wt") as f:
    PDBFile.writeFile(fixer.topology, fixer.positions, f)

In [10]:
import nglview

w = nglview.show_structure_file(fnout)
w.add_representation("line")
w

NGLWidget()